In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import *
try:
    stopwords = set(stopwords.words('english'))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stopwords = set(stopwords.words('english'))
#stopwords
import re, nltk, spacy, gensim
from google.colab import drive

In [0]:
from google.colab import auth
auth.authenticate_user()

from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

myfile = drive.CreateFile({'id': ''})
#df=pd.read_csv('gdrive/My Drive/ColabNotebooks/sample_data/googleplaystore_user_reviews.csv')
myfile.GetContentFile('snli_1.0_test.csv')

In [0]:
import pandas as pd
#frm = pd.read_csv('googleplaystore_user_reviews.csv', header=None)
df = pd.read_csv('snli_1.0_test.csv')
df = df.dropna(subset=['sentence1'])
df.head()

In [0]:
# Convert to list
data = df.sentence1.values.tolist()
# Remove 
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
# Remove https 
data = [re.sub(r'http\S+', '', sent) for sent in data]

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

In [0]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [0]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) #select noun and verb
print(data_lemmatized[:1])

In [0]:
import nltk
import itertools
nltk.download('punkt')
from nltk import FreqDist
#from nltk import FreqDis
from nltk import word_tokenize 
from nltk.util import ngrams

In [0]:
def create_graph(data):
  global  window_counter, af,model, dict_test, tau, t,asz
  # Combine all training text into one large string
  all_text1 = (data_lemmatized)
  edges1 = []
  ah = [ascii]
  window_counter = 0
  lstt = []
  see = 0
  for line in all_text1:
    token = nltk.word_tokenize(line)
    bigram = list(ngrams(token, 2))
    edges1.append(bigram)
    edges = list(itertools.chain(*edges1))

  return edges

In [0]:
bigram_edges_graph = create_graph(data)
#print(bigram_edges_graph)
print(len(bigram_edges_graph))

In [0]:
import time
import json
import datetime
import asyncio
import random
import sys
import csv
import requests
#@title Capture Twitter streaming (double-click to view) {display-mode: "form"}

k =  200#@param {type:"number"}
#tau = 3 #@param {type:"number"}
#lamb = 1 #@param {type:"number"}
#tracking = "bitcoin" #@param {type:"string"}
#threshold = 5 #@param {type:"slider", min:1, max:20, step:1}
#timeout = 5 #@param {type:"slider", min:1, max:120, step:1}
#imeout = int(timeout)*60000
#start = int(round(time.time() * 1000))

global L,see, sample_window_stream, global_counter, time_counter, interval_counter,m,kr
#L=0
#sample_window_stream = []
#global_counter = 0
#interval_counter = 0
#time_counter = (time.time()*1000)+60000


# k is the size of the reservoir (number of edges into the reservoir)


# tau is the lenght of the window
#tau =  tau*60000

# lamb is the lenght of strate floato the window
#lamb = lamb*60000

# rate is the number of strates per window
#rate = tau/lamb

# M contain steps floato the window, for each strates we have a edges counter
#M = [0] * int(rate)

# init it's a default parameters to configure somes variables when the reservoir strart
init = 0

reservoir = []


def step_reservoir_sampling(edge):
    global  model, dict_test, tau, t,m,kr
    #m = 0
    #for token in bigram_edges_graph:
    if m<=k:
      reservoir.append(edge)
    if m>k and p<=1: 
      j = random.randint(0, 1)
      if j <= p:
        kr = random.randint(1,k)
        del reservoir[kr-1]
        reservoir.append(edge)
      if m>k  and p>1:
        del reservoir[kr]
        reservoir.append(edge)
      #m +=1
    
    return reservoir
    #if j < k:
     # if len(reservoir) < k:
      #  reservoir.append(edge)
       # print(reservoir)
      #else:
       # del reservoir[j]
        #reservoir.append(edge)
    #return reservoir

#for edge in edges:
#  window_reservoir_sampling = step_reservoir_sampling(edge)

#print(window_reservoir_sampling)

In [0]:
import string
import gensim
from gensim.models import phrases, word2vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

matplotlib.style.use('ggplot')

%matplotlib inline
#model = word2vec.Word2Vec(bigram, size=50, min_count=3, iter=20)
model = word2vec.Word2Vec(bigram_edges_graph, min_count=1)

In [0]:
global  model,m
ah = [ascii]
window_counter = 0
lstt = []
see = 0
m = 0
for edge in bigram_edges_graph:
    #se = (edge[0], edge[1]),model.wv.similarity(edge[0], edge[1])
    ah.append(model.wv.similarity(edge[0], edge[1]))
    ww = abs(model.wv.similarity(edge[0], edge[1]))
    see += ww
    #print(see)
    #see = see + ww
    #p = (ww*k/see)
    p = (ww*k)/see
    #print (p)
    #lstt.append(ww)
      #print (lstt)
      #print(wsss)
    window_reservoir_sampling = step_reservoir_sampling(edge)
    m +=1
    window_counter += 1
    
    #see = lstt[(window_counter - 1)]
print(window_reservoir_sampling)
print(len(window_reservoir_sampling))
#print(ah)

In [0]:
# Set the dictionary d

d={}
n=0
for e in window_reservoir_sampling:
   a=e[0]
   b=e[1]
   if a in d.keys(): d[a].add(b)
   else: d[a]=set([b])
   if b in d.keys(): d[b].add(a)
   else: d[b]=set([a])
   
   
#print ("Dict=", d, len(d))
n=len(d)
m=0
#Breadth-first search from first point, first component
a=list(d.keys())[0]

#dc keeps b:i  if i is the length of the shortest path from a to b in the first component
dc={}
dc[a]=0
for b in d[a]: 
  if b in dc.keys(): pass
  else: dc[b]=1

#Initialize S ans S1, Start iterating
S=d[a]
comp=[]
S1=set([a])
S=S.union(S1)
#print("S=",S)
while S > S1:
    S1=S
    for u in S:
       S=S.union(d[u])
       for v in d[u]:
         if v in dc.keys(): dc[v]=min(dc[v],dc[u]+1)
         else: dc[v]=dc[u]+1
for u in S:
     m=m+len(d[u])

comp.append((len(S),int(m/2),list(S)))
  
#print("Component",comp)

ST=S
  
#print("ST=",ST)

#The other components: origin must be outside ST, same treatment
i=1
while i<len(d):
 m=0
 while  list(d.keys())[i] not in ST:
  a=list(d.keys())[i]
  S=d[a]
  S1=set([a])
  S=S.union(S1)
  while S > S1:
    S1=S
    for u in S:
       S=S.union(d[u])
  for u in S:
     m=m+len(d[u])
  comp.append((len(S),int(m/2),list(S)))
  ST=ST.union(S)  
 i+=1     
   
comp1=sorted(comp,reverse=True)    
print("Components",comp1)
#print("Diameter of largest component  =",diameter(0))
#To compute the diameter of  component i, do diameter(i)
#List of edges of each component
#Go through comp1 then the list of nodes and with the dictionary generate the edges

compedges=[]
j0=0
while j0<len(comp1):
  l1 = comp1[j0][2]
#  print(l1, len(l1))
  j=0
  cp=[]
  while j <len(l1):
    a=l1[j]
#  print(d[l1[j]])
    j1=0
    while j1< len(d[l1[j]]):
  	  f=(a,list(d[l1[j]])[j1])
  	  cp.append(f)
  	  j1 +=1

    j +=1
  j0 +=1
  compedges.append(cp)
print("Edges of each component",compedges)

In [0]:
print("Components",comp1[0])